In [1]:
import sys
sys.path.append("../models")
sys.path.append("../utils")
sys.path.append("../trainer")
import config
import time
import numpy as np
import tensorflow as tf
import data_loader
from tra_helper.image_aug import img_missing
from tra_helper.plot_dset_one import plot_dset_one
from seg_models.model_triple import model_triple
from seg_models.unet import unet
from base_models.mobilenet import mobilenet_v2


In [2]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [3]:
root_dir = '/home/yons/Desktop/developer-luo/SWatNet'
## 1. dataset
tra_dset = data_loader.get_tra_dset()
test_dset = data_loader.get_eva_dset()
tra_dset_dist = strategy.experimental_distribute_dataset(tra_dset)
test_dset_dist = strategy.experimental_distribute_dataset(test_dset)

## 2. training configuration
with strategy.scope():
    loss_fun = tf.keras.losses.BinaryCrossentropy(
        from_logits=False,
      reduction=tf.keras.losses.Reduction.NONE)
    def compute_loss(labels, predictions):
        per_example_loss = loss_fun(labels, predictions)
        # per_example_loss /= tf.cast(tf.reduce_prod(tf.shape(labels)[1:]), tf.float32)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=config.BATCH_SIZE)

### metrics
with strategy.scope():
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    train_accuracy = tf.keras.metrics.BinaryAccuracy('tra_oa')
    test_accuracy = tf.keras.metrics.BinaryAccuracy('test_oa')

# optimizer = config.optimizer
# loss_fun = config.binary_ce_loss
# tra_loss_tracker = config.tra_loss_tracker
# tra_oa = config.tra_oa
# tra_miou = config.tra_miou


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [4]:
# with strategy.scope():
#     # model = model_triple(input_shape=(256,256,4), base_model=mobilenet_v2)
#     model = unet(nclass=2)
#     optimizer = tf.keras.optimizers.Adam()

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=1, kernel_size=1),
        # tf.keras.layers.Dense(1)
        ])
    optimizer = tf.keras.optimizers.Adam()


In [5]:
def train_step(x,y):
    with tf.GradientTape() as tape:
        y_pre = model(x, training=True)
        loss = loss_fun(y, y_pre)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_accuracy.update_state(y, y_pre)
    return loss

@tf.function
def distributed_train_step(x,y):
    per_replica_losses = strategy.run(train_step, args=(x,y,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


In [6]:
for epoch in range(10):
    print(epoch)
    for x_batch, y_batch in test_dset_dist:
        # x_batch = img_missing(x_batch)
        x_batch=x_batch[2]   ##!!note: x_batch[2] for single-scale model
        total_loss = distributed_train_step(x_batch,y_batch)
    print('loss:', total_loss.result())


0
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1


ValueError: in user code:

    <ipython-input-5-66da89dd14a6>:13 distributed_train_step  *
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1371 reduce  **
        return self._extended._reduce(reduce_op, value)  # pylint: disable=protected-access
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2214 _reduce
        return self._local_results(self.reduce_to(reduce_op, value, dst))[0]
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2299 reduce_to
        return self._reduce_to(reduce_op, value, destinations, options)
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/mirrored_strategy.py:684 _reduce_to
        options=

In [21]:
'''------train step------'''
# @tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pre = model(x, training=True)
        loss = compute_loss(y, y_pre)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_accuracy.update_state(y, y_pre)
    return loss

@tf.function
def distributed_train_step(x, y):
    per_replica_losses = strategy.run(train_step, args=(x, y))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

'''------test step------'''
# @tf.function
def test_step(x, y):
    y_pre = model(x, training=False)
    loss = loss_fun(y, y_pre)
    test_loss.update_state(loss)
    test_accuracy.update_state(y, y_pre)
    return test_accuracy.result()

@tf.function
def distributed_test_step(x, y):
    return strategy.run(test_step, args=(x, y))

'''------train loops------'''
def train_loops(tra_dset, test_dset, epochs):
    max_miou_pre = 0.8
    num_batches = 0
    for epoch in range(epochs):
        print(epoch)
        start = time.time()

        '''---train the model---'''
        for x_batch, y_batch in tra_dset:
            # x_batch = img_missing(x_batch)
            x_batch=x_batch[2]   ##!!note: x_batch[2] for single-scale model
            total_loss = distributed_train_step(x_batch, y_batch)
            num_batches += 1
        train_loss = total_loss/num_batches

        '''---test the model---'''
        for x_batch, y_batch in test_dset:
            x_batch=x_batch[2]  ##!note: x_batch[2] for single-scale model
            distributed_test_step(x_batch, y_batch)

        template = ("Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, "
              "Test Accuracy: {:.3f}, Time: {:.1f}")
        print(template.format(epoch+1, train_loss, train_accuracy.result(),
            test_loss.result(), test_accuracy.result(),  time.time() - start))
        
        test_loss.reset_states()
        train_accuracy.reset_states()
        test_accuracy.reset_states()

        '''---visualize the results---'''
        if epoch%20 == 0:
            figure = plot_dset_one(model, test_dset.take(1), \
                        i_patch=np.random.randint(8), binary=False, \
                        multiscale=False, weight=False)


In [22]:
## training
train_loops(tra_dset_dist, test_dset_dist, epochs=200)



0
INFO:tensorflow:batch_all_reduce: 118 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 118 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Error reported to Coordinator: 
Traceback (most recent call last):
  File "/home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/mirrored_run.py", line 210, in _call_for_each_replica
    tuple(t.merge_args for t in threads))
  File "/home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_utils.py", line 62, in regroup
    for i in range(len(v0)))
  File "/home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_utils.py", line 62, in <genexpr>
    for i in range(len(v0)))
 

AssertionError: in user code:

    <ipython-input-15-37ed38b9cf67>:14 distributed_train_step  *
        per_replica_losses = strategy.run(train_step, args=(x, y))
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/mirrored_strategy.py:629 _call_for_each_replica
        self._container_strategy(), fn, args, kwargs)
    /home/yons/Desktop/developer-luo/SWatNet/venv/lib/python3.6/site-packages/tensorflow/python/distribute/mirrored_run.py:93 call_for_each_replica
        return _call_for_each_replica(strategy,